## The objective of this notebook is to compare several similarity metrics, in the area of biocollections, and envisioning the construction process of a similarity graph

### Essential answers (from Dr. Fortes):
- % of records with certain % of similarity
- % of records with certain % of similarity considering n fields

### Some implementation reasoning:
- When we talk about similarity, we assume the existence of two strings or a ground true value which is compared to other test values. In the case of biocollection metadata, what to we want to compare to what?
- A cross product of all the values of all the terms compared to all the other values seem too costly, computationally.
- The similarity that is computed in this notebook does not have any biological knowledge: we do not consider similarity among specimens' families or genus or something similar. This is a general or ignorant procedure, considering the semantic value of the data.

### Probabilistic databases: 
I checked some documentation about probabilistic databases, because they could be a good storage option for the processed data, but the available software I found is outdated or not mature enough. 
Moreover, most of these databases include probability values for the tuple (tuple-level uncertainty) or for some column (attribute-level uncertainty), not for the relations among values.

## Similarity analysis of the Scientific name

In [1]:
import pandas as pd
from pyspark.sql.functions import count, col

In [2]:
dataset = spark.read.parquet("./preston-amazon/data-processed/core.parquet")
dataset = dataset.fillna('')
N = dataset.count()
print("Number of rows: " + str(N))

Number of rows: 3858


In [3]:
df = pd.DataFrame( columns = ['sc_name', 'count'] )


In [4]:
dataset_sn = dataset.groupBy("`http://rs.tdwg.org/dwc/terms/scientificName`").count().select(col("`http://rs.tdwg.org/dwc/terms/scientificName`").alias('sc_name'), col('count').alias('n'))

In [5]:
dataset_sn.count()
# Number of different scientific names

708

This means that for our collection of 3,858 records, the complete scientific name analysis would imply (708 x 708 / 2) - 708 = 249,924 comparisons.
We should think about questions like:
- Is it meaningful to conclude that a scientific name is 65% similar to another one?
- Is it meaningful to conclude that a date is 70% similar to another one?

In [6]:
dataset_sn.orderBy('count', ascending=False).show(10, False)

+-------------------------------------------+---+
|sc_name                                    |n  |
+-------------------------------------------+---+
|Triportheus angulatus (Spix & Agassiz 1829)|51 |
|Serrasalmus rhombeus (Linnaeus 1766)       |49 |
|Pygocentrus nattereri Kner 1858            |49 |
|Pimelodus blochii Valenciennes 1840        |48 |
|Sorubim lima (Bloch & Schneider 1801)      |45 |
|Ageneiosus inermis (Linnaeus 1766)         |43 |
|Moenkhausia dichroura (Kner 1858)          |43 |
|Roeboides affinis (Günther 1868)           |42 |
|Schizodon fasciatus Spix & Agassiz 1829    |41 |
|Acestrorhynchus gr. lacustris (Lütken 1875)|40 |
+-------------------------------------------+---+
only showing top 10 rows



In [7]:
sn_list = dataset_sn.select('sc_name').rdd.flatMap(lambda x: x).collect()

In [8]:
print(sn_list[0])

Arocera (Euopta) placens Walker 1867


In [9]:
from pyxdameraulevenshtein import normalized_damerau_levenshtein_distance

In [10]:
%%time
#subset = sn_list[0:100]
subset = sn_list
i = 0
j = 0
while i < len(subset):
    j = i+1
    while j < len(subset):
        if (i != j):
            sim = 1 - normalized_damerau_levenshtein_distance (subset[i], subset[j])
            if (sim > 0.9):
                print( str(sim) + "\t" + subset[i] + " - " + subset[j] )
        j = j + 1
    i = i + 1

0.9459459446370602	Rio indistinctus Fortes & Grazia 2000 - Rio distinctus Fortes & Grazia 2000
0.9019607827067375	Edessa rufodorsata Silva, Fernandes & Grazia 2006 - Edessa virididorsata Silva, Fernandes & Grazia 2006
0.9677419364452362	Gonatopus neotropicus Olmi 1984 - Gonatopus neotropicus Olmi 1986
CPU times: user 13.8 s, sys: 0 ns, total: 13.8 s
Wall time: 13.8 s


In short terms, like scientific name, the Damerau Levenshtein similarity may be more useful than using n-grams.
To have a better idea about how to implement it and how to present the results, I believe we should discuss: how do we expect these results are going to be used? 

## Utilization of the Edit (D-L) Distance and Jaccard Distance with NLTK to generate Similarity values

In [11]:
import nltk

In [17]:
%%time
# subset = sn_list[0:4]
subset = sn_list
i = 0
j = 0
while i < len(subset):
    j = i+1
    while j < len(subset):
        if (i != j):
            max_length = len(subset[i])
            if len(subset[j]) > max_length:
                max_length = len(subset[j])
            if max_length > 0:
                distance = nltk.edit_distance( subset[i], subset[j] )
                sim = 1 - (distance/max_length)
                if (sim > 0.9):
                    print( str(sim) + "\t" + subset[i] + " - " + subset[j] )
        j = j + 1
    i = i + 1

0.9459459459459459	Rio indistinctus Fortes & Grazia 2000 - Rio distinctus Fortes & Grazia 2000
0.9019607843137255	Edessa rufodorsata Silva, Fernandes & Grazia 2006 - Edessa virididorsata Silva, Fernandes & Grazia 2006
0.967741935483871	Gonatopus neotropicus Olmi 1984 - Gonatopus neotropicus Olmi 1986
CPU times: user 2min 50s, sys: 0 ns, total: 2min 50s
Wall time: 2min 50s


### Notes:
- Results and values are the same in both libraries, with differences after the eight decimal position.
- The NLTK implementation is much slower: 170s vs. 14s (~12x).
- NLTK does not provide a normalized DL distance. That is why I had to implement it.

## Edit distance (Damerau-Levenshtein):
"The edit distance is the number of characters that need to be substituted, inserted, or deleted, to transform s1 into s2.  For example, transforming "rain" to "shine" requires three steps, consisting of two substitutions and one insertion: "rain" -> "sain" -> "shin" -> "shine".  These operations could have been done in other orders, but at least three steps are needed."

Taken from: https://www.nltk.org/_modules/nltk/metrics/distance.html

## Jaccard Distance:
Jaccard Distance depends on another concept called “Jaccard Similarity Index” which is (the number in both sets) / (the number in either set)

J(X,Y) = |X∩Y| / |X∪Y|

Then we can calculate the Jaccard Distance as follows:

D(X,Y) = 1 – J(X,Y)

For example, if we have two strings: “mapping” and “mappings”, the intersection of the two sets is 6 because there are 7 similar characters, but the “p” is repeated while we need a set, i.e. unique characters, and the union of the two sets is 7, so the Jaccard Similarity Index is 6/7 = 0.857 and the Jaccard Distance is 1 – 0.857 = 0.142

Taken from: https://python.gotrained.com/nltk-edit-distance-jaccard-distance/ 

In [22]:
%%time
# subset = sn_list[0:100]
subset = sn_list
i = 0
j = 0
while i < len(subset):
    j = i+1
    while j < len(subset):
        if (i != j):
            sim = 1 - nltk.jaccard_distance( set(subset[i]), set(subset[j]) )
            if (sim > 0.9):
                print( str(sim) + "\t" + subset[i] + " - " + subset[j] )
        j = j + 1
    i = i + 1

0.9090909090909091	Arocera (Euopta) placens Walker 1867 - Arocera (Euopta) apta Walker 1867
1.0	Rio indistinctus Fortes & Grazia 2000 - Rio distinctus Fortes & Grazia 2000
0.9444444444444444	Apenesia truncaticeps Kieffer 1910 - Apenesia crenulata Kieffer 1910
0.9411764705882353	Edessa corallipes Erichson 1848 - Edessa alces Erichson 1848
0.9047619047619048	Apenesia rostrum Azevedo & Batista 2002 - Apenesia megaventris Azevedo & Batista 2002
0.9523809523809523	Elanela rafaeli Grazia, Barros & Barao 2016 - Elanela fernandes Grazia, Barros & Barao 2016
0.9047619047619048	Edessa xingu Fernandes & van Doesburg 2000 - Edessa guyanensis Fernandes & van Doesburg 2000
0.95	Edessa xingu Fernandes & van Doesburg 2000 - Edessa rondoniensis Fernandes & van Doesburg 2000
0.95	Edessa guyanensis Fernandes & van Doesburg 2000 - Edessa rondoniensis Fernandes & van Doesburg 2000
0.9047619047619048	Antiteuchus schuhi Engleman 1983 - Antiteuchus geometricus Engleman 1983
0.9047619047619048	Antiteuchus schu

### Notes:
- In Jaccard, a similarity of 1.0 does not mean the strings are the same. (apple = plea)
- Jaccard tends to give higher scores, is more relaxed in some sense, but that also permits to retrieve more candidates (of course, we could decrease the acceptance threshold at DL). At the end, it seems a matter of preference.